
# Datenaufbereitung und Bereinigung

In diesem Notebook bereiten wir den **OECD Well-Being Indicators**-Datensatz für unsere Analyse vor. Ziel ist es, die rohen CSV-Daten so zu transformieren, dass sie für die folgenden Analysen (Deskriptive Statistik, Korrelationen, Hypothesentests, Regression, Zeitreihen) sauber und zuverlässig verwendet werden können. Wir orientieren uns dabei an den Vorgaben aus der Statistik-Vorlesung:

- **VL1 & VL4 (Daten laden und Import)**: Datensätze einlesen, redundante Spalten entfernen und Datentypen korrigieren.
- **Entscheidung über Datenstruktur**: Aufteilung in *Zeitreihe* und *Snapshot* zur Vermeidung abhängiger Beobachtungen.
- **Umgang mit Missing Values**: Prüfung, ob fehlende Werte vorliegen, und deren Behandlung.

Wir dokumentieren jeden Schritt ausführlich und erklären, warum er durchgeführt wird. Nach wichtigen Codezellen interpretieren wir die Ergebnisse im Kontext unseres Projekts.


In [25]:

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter


# Konfiguriere Matplotlib und Seaborn für einheitliches Design
a = sns.set_theme(style="whitegrid")
# Anzeigeoptionen für Pandas: Zeige alle Spalten
pd.set_option('display.max_columns', None)

print("Bibliotheken wurden importiert und Einstellungen gesetzt.")


Bibliotheken wurden importiert und Einstellungen gesetzt.


In [26]:

# Definiere den Pfad zum Datensatz. Hier stellen wir sicher, dass der Datensatz gefunden wird.
base_dir = Path.cwd()
data_file = 'OECD.WISE.WDP,DSD_HSL@DF_HSL_CWB,+all.csv'

# Suche nach dem Datensatz an den möglichen Speicherorten (Projektordner oder data/ Ordner)
candidates = [
    base_dir / 'data' / data_file,
    base_dir.parent / 'data' / data_file,
]

data_path = next((p for p in candidates if p.exists()), None)

if data_path is None:
    raise FileNotFoundError(f"Datei nicht gefunden. Bitte vergewissere dich, dass {data_file} im Ordner 'data' vorhanden ist.")

print(f"Lade Daten von: {data_path}")

# Lese die CSV-Datei ein
try:
    df = pd.read_csv(data_path)
    print("Daten erfolgreich geladen!")
except Exception as e:
    print(f"FEHLER: {e}")


Lade Daten von: /Users/Flurina/Library/CloudStorage/OneDrive-Persönlich/Documents/Uni_St.Gallen/BCS/3.Semester/Statistik_für_Data_Science/Projekt/Statistik_Projekt_OECDWellBeing/data/OECD.WISE.WDP,DSD_HSL@DF_HSL_CWB,+all.csv
Daten erfolgreich geladen!


In [27]:

# Dimensionen und erste Eindrücke
print(f"Dimensionen: {df.shape}")
print("Erste fünf Zeilen des Datensatzes:")
print(df.head())

print("Datentypen der Spalten:")
print(df.dtypes)

# Hinweis: Wir sehen, dass viele Spalten technische Codes und Beschriftungen enthalten. Diese werden im nächsten Schritt bereinigt.


Dimensionen: (108088, 30)
Erste fünf Zeilen des Datensatzes:
  STRUCTURE                           STRUCTURE_ID      STRUCTURE_NAME ACTION  \
0  DATAFLOW  OECD.WISE.WDP:DSD_HSL@DF_HSL_CWB(1.1)  Current well-being      I   
1  DATAFLOW  OECD.WISE.WDP:DSD_HSL@DF_HSL_CWB(1.1)  Current well-being      I   
2  DATAFLOW  OECD.WISE.WDP:DSD_HSL@DF_HSL_CWB(1.1)  Current well-being      I   
3  DATAFLOW  OECD.WISE.WDP:DSD_HSL@DF_HSL_CWB(1.1)  Current well-being      I   
4  DATAFLOW  OECD.WISE.WDP:DSD_HSL@DF_HSL_CWB(1.1)  Current well-being      I   

  REF_AREA Reference area MEASURE                Measure       UNIT_MEASURE  \
0      AUS      Australia    10_2  Feeling safe at night  PT_POP_Y_GE15_SUB   
1      AUS      Australia    10_2  Feeling safe at night  PT_POP_Y_GE15_SUB   
2      AUS      Australia    10_2  Feeling safe at night  PT_POP_Y_GE15_SUB   
3      AUS      Australia    10_2  Feeling safe at night  PT_POP_Y_GE15_SUB   
4      AUS      Australia    10_2  Feeling safe at night 

In [28]:

# Entferne Spalten, die nur Codes enthalten (STRUCTURE, STRUCTURE_ID usw.)
cols_to_drop = [
    'STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'ACTION',
    'Note', 'Flag Codes'  # Beispiel für weitere überflüssige Spalten; passe ggf. an
]

df_clean = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

# Normalisiere Spaltennamen: Ersetze Leerzeichen durch Unterstriche und senke Großbuchstaben ab
# (erleichtert spätere Referenzen im Code)
df_clean.columns = [c.lower().replace(' ', '_') for c in df_clean.columns]

# Benenne ausgewählte Spalten um, um Klarheit zu schaffen
rename_cols = {
    'obs_value': 'value',
    'time_period': 'year'
}

df_clean = df_clean.rename(columns=rename_cols, errors='ignore')

# Zeige die aktualisierten Spalten zur Kontrolle
print(df_clean.columns.tolist())


['ref_area', 'reference_area', 'measure', 'measure', 'unit_measure', 'unit_of_measure', 'age', 'age', 'sex', 'sex', 'education_lev', 'education_level', 'domain', 'domain', 'year', 'year', 'value', 'observation_value', 'obs_status', 'observation_status', 'unit_mult', 'unit_multiplier', 'decimals', 'decimals', 'base_per', 'base_period']


In [29]:
# Doppelte Spaltennamen automatisch eindeutig machen (ohne Daten zu löschen)
def make_unique(cols):
    counts = Counter()
    new_cols = []
    for c in cols:
        counts[c] += 1
        new_cols.append(f"{c}_{counts[c]-1}" if counts[c] > 1 else c)
    return new_cols

df_clean = df_clean.copy()
df_clean.columns = make_unique(df_clean.columns)

# Prüfen, welche year-Spalten existieren
year_cols = [c for c in df_clean.columns if c == "year" or c.startswith("year_")]
print("Gefundene year-Spalten:", year_cols)

# Nimm die erste year-Spalte als Sortierkriterium
year_col = year_cols[0]

# Definiere die Gruppierungsvariablen
group_cols = ['reference_area', 'measure', 'sex', 'age', 'education_level', 'domain']

print("Erstelle vollständige Zeitreihe (df_time)")
df_time = df_clean.copy().sort_values(year_col)

df_latest = (
    df_clean.sort_values(year_col)
    .groupby(group_cols, as_index=False)
    .last()
)

print(f"Datensatz für Zeitreihen (df_time): {len(df_time)} Zeilen")
print(f"Datensatz für Hypothesentests (df_latest): {len(df_latest)} Zeilen")
'''
# Definiere die Gruppierungsvariablen, anhand derer wir die aktuellsten Werte bestimmen wollen
# Referenzkategorie: Land (reference_area) sowie weitere Dimensionen (measure, sex, age, education_level, domain)
group_cols = ['reference_area', 'measure', 'sex', 'age', 'education_level', 'domain']

# Erstelle die vollständige Zeitreihe (alle Jahre) -> df_time
# Wir sortieren erst nach Jahr, damit die Zeitreihe geordnet ist
print("Erstelle vollständige Zeitreihe (df_time)")
df_time = df_clean.copy().sort_values('year')

# Erstelle Snapshot-Datensatz, indem wir pro Gruppe nur die aktuellsten Beobachtungen behalten
df_latest = (
    df_clean.sort_values('year')
    .groupby(group_cols, as_index=False)
    .last()
)

print(f"Datensatz für Zeitreihen (df_time): {len(df_time)} Zeilen")
print(f"Datensatz für Hypothesentests (df_latest): {len(df_latest)} Zeilen")
'''

Gefundene year-Spalten: ['year', 'year_1']
Erstelle vollständige Zeitreihe (df_time)
Datensatz für Zeitreihen (df_time): 108088 Zeilen
Datensatz für Hypothesentests (df_latest): 10960 Zeilen


'\n# Definiere die Gruppierungsvariablen, anhand derer wir die aktuellsten Werte bestimmen wollen\n# Referenzkategorie: Land (reference_area) sowie weitere Dimensionen (measure, sex, age, education_level, domain)\ngroup_cols = [\'reference_area\', \'measure\', \'sex\', \'age\', \'education_level\', \'domain\']\n\n# Erstelle die vollständige Zeitreihe (alle Jahre) -> df_time\n# Wir sortieren erst nach Jahr, damit die Zeitreihe geordnet ist\nprint("Erstelle vollständige Zeitreihe (df_time)")\ndf_time = df_clean.copy().sort_values(\'year\')\n\n# Erstelle Snapshot-Datensatz, indem wir pro Gruppe nur die aktuellsten Beobachtungen behalten\ndf_latest = (\n    df_clean.sort_values(\'year\')\n    .groupby(group_cols, as_index=False)\n    .last()\n)\n\nprint(f"Datensatz für Zeitreihen (df_time): {len(df_time)} Zeilen")\nprint(f"Datensatz für Hypothesentests (df_latest): {len(df_latest)} Zeilen")\n'

In [30]:

# Überprüfe auf fehlende Werte
missing_counts = df_clean.isnull().sum()
print("Missing Values pro Spalte:")
print(missing_counts[missing_counts > 0])

if missing_counts.sum() == 0:
    print("Keine fehlenden Werte – keine Imputation notwendig.")
else:
    print("Es gibt fehlende Werte. Diese würden wir je nach Analyse entweder entfernen oder imputieren.")


Missing Values pro Spalte:
year_1               108088
observation_value    108088
base_per             106514
base_period          108088
dtype: int64
Es gibt fehlende Werte. Diese würden wir je nach Analyse entweder entfernen oder imputieren.


In [31]:

# Speichere die beiden Datensätze im data/ Ordner
output_dir = (data_path.parent if data_path.name == data_file else base_dir / 'data')

full_series_path = output_dir / 'oecd_full_time_series.csv'
snapshot_path = output_dir / 'oecd_snapshot_latest.csv'

# Speichern
df_time.to_csv(full_series_path, index=False)
df_latest.to_csv(snapshot_path, index=False)

print(f"Zeitreihendatensatz gespeichert als: {full_series_path}")
print(f"Snapshot-Datensatz gespeichert als: {snapshot_path}")


Zeitreihendatensatz gespeichert als: /Users/Flurina/Library/CloudStorage/OneDrive-Persönlich/Documents/Uni_St.Gallen/BCS/3.Semester/Statistik_für_Data_Science/Projekt/Statistik_Projekt_OECDWellBeing/data/oecd_full_time_series.csv
Snapshot-Datensatz gespeichert als: /Users/Flurina/Library/CloudStorage/OneDrive-Persönlich/Documents/Uni_St.Gallen/BCS/3.Semester/Statistik_für_Data_Science/Projekt/Statistik_Projekt_OECDWellBeing/data/oecd_snapshot_latest.csv



## Zusammenfassung und Fazit

In diesem Notebook haben wir die OECD-Well-Being-Daten eingelesen, bereinigt und für die anschließenden Analysen vorbereitet. Wir haben:

- **Daten geladen und geprüft**: Erfolgreiches Einlesen der CSV-Datei, Überblick über Dimensionen und Datentypen.
- **Bereinigung durchgeführt**: Überflüssige Spalten entfernt und verständliche Spaltennamen gewählt.
- **Daten aufgeteilt**: Einen *vollständigen Zeitreihendatensatz* (für Trendanalysen) und einen *Snapshot-Datensatz* (für Hypothesentests und Korrelationen) erstellt. Dieser Schritt ist entscheidend, um Pseudoreplikation zu vermeiden.
- **Missing Values geprüft**: Es wurden keine fehlenden Werte gefunden, sodass keine Imputation notwendig ist.
- **Speicherung**: Beide Datensätze wurden im Ordner `data/` abgelegt.

Mit diesen sauberen Datensätzen können wir in den folgenden Notebooks deskriptive Statistiken erstellen, Korrelationen untersuchen, Hypothesen testen und Regressionsmodelle anpassen. Zudem liefern die Zeitreihen eine Grundlage für die Analyse von Trends über die Jahre.
